CELL 1 — Paths & Load Silver Data

In [ ]:
storage = "your-storage-name"

silver_products = f"abfss://datalake@{storage}.dfs.core.windows.net/silver/fakestore/products/"
silver_users    = f"abfss://datalake@{storage}.dfs.core.windows.net/silver/fakestore/users/"
silver_orders   = f"abfss://datalake@{storage}.dfs.core.windows.net/silver/fakestore/orders/"
silver_order_details = f"abfss://datalake@{storage}.dfs.core.windows.net/silver/fakestore/order_details/"

df_prod = spark.read.parquet(silver_products)
df_users = spark.read.parquet(silver_users)
df_orders = spark.read.parquet(silver_orders)


StatementMeta(spkecommerce, 9, 2, Finished, Available, Finished)

In [4]:
# Join orders with products using product_id
orders_products = df_orders.join(
    df_prod,
    on="product_id",
    how="left"
)

# Join the result with users using user_id
orders_products_users = orders_products.join(
    df_users,
    on="user_id",
    how="left"
)

# Show schema after both joins
orders_products_users.printSchema()


StatementMeta(spkecommerce, 9, 5, Finished, Available, Finished)

root
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- cart_id: integer (nullable = true)
 |-- cart_date: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- order_ts: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- description: string (nullable = true)
 |-- image: string (nullable = true)
 |-- price: double (nullable = true)
 |-- rating: struct (nullable = true)
 |    |-- count: long (nullable = true)
 |    |-- rate: double (nullable = true)
 |-- title: string (nullable = true)
 |-- __v: long (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- geolocation: struct (nullable = true)
 |    |    |-- lat: string (nullable = true)
 |    |    |-- long: string (nullable = true)
 |    |-- number: long (nullable = true)
 |    |-- street: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |-- email

In [5]:
from pyspark.sql.functions import col, concat_ws

df_order_details = orders_products_users.select(
    # Order info
    col("cart_id").alias("order_id"),
    col("order_ts").alias("order_timestamp"),
    col("cart_date").alias("order_date_raw"),

    # User info
    col("user_id"),
    concat_ws(" ", col("name.firstname"), col("name.lastname")).alias("customer_name"),
    col("email").alias("customer_email"),
    col("phone").alias("customer_phone"),
    col("address.city").alias("customer_city"),

    # Product info
    col("product_id"),
    col("title").alias("product_title"),
    col("category").alias("product_category"),
    col("price").alias("product_price"),

    # Order amounts
    col("quantity"),
    (col("price") * col("quantity")).alias("total_amount"),

    # Metadata
    col("ingest_ts")
)

df_order_details.printSchema()
display(df_order_details.limit(20))


StatementMeta(spkecommerce, 9, 6, Finished, Available, Finished)

root
 |-- order_id: integer (nullable = true)
 |-- order_timestamp: timestamp (nullable = true)
 |-- order_date_raw: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- customer_name: string (nullable = false)
 |-- customer_email: string (nullable = true)
 |-- customer_phone: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)



SynapseWidget(Synapse.DataFrame, 2d6268b9-22e6-4371-8dd1-2daf4c978382)

In [6]:
df_order_details.write.mode("overwrite").parquet(silver_order_details)
print("Silver OrderDetails fact table created successfully!")


StatementMeta(spkecommerce, 9, 7, Finished, Available, Finished)

Silver OrderDetails fact table created successfully!
